In [1]:
from DataSplit import load_data
from Tokenizer import tokenize

X_train, X_test, X_eval, y_train, y_test, y_eval = load_data()
print(len(X_train))
print(len(X_test))
print(len(X_eval))
print(y_train[0])


tokenize(X_train) # Basic tokenizer that removes punctuation and lowercases everything
tokenize(X_test)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ellio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ellio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


3200
400
400
1


In [2]:
from Tokenizer import remove_stopwords_and_stem
from Tokenizer import remove_stopwords_and_lemmatize
from FrequencyCut import cut
from PoSTagging import run
import numpy as np

# edit the stopwords to not remove the word "not"

for x in range(len(X_train)):
    X_train[x] = remove_stopwords_and_stem(X_train[x])
    # X_train[x] = remove_stopwords_and_lemmatize(X_train[x])

for x in range(len(X_test)):
    X_test[x] = remove_stopwords_and_stem(X_test[x])
    # X_test[x] = remove_stopwords_and_lemmatize(X_test[x])


# get all the words in the training set

# X_train = cut(X_train)

postagstrain = run(X_train)
min = np.min(postagstrain)
max = np.max(postagstrain)
postagstrain = (postagstrain - min) / (max - min) #normalise

postagstest = run(X_test)
min = np.min(postagstest)
max = np.max(postagstest)
postagstest = (postagstest - min) / (max - min) #normalise



print(X_train[0])

['sur', 'cheesy', 'nonsens', 'tim', 'corny', 'least', 'filmmak', 'didnt', 'try', 'tv', 'movy', 'bord', 'brink', 'mediocr', 'film', 'act', 'redeem', 'qual', 'cinematograph', 'pretty', 'good', 'tv', 'film', 'viggo', 'mortens', 'display', 'shad', 'aragorn', 'film', 'man', 'play', 'rul', 'flashback', 'sequ', 'kind', 'cheesy', 'scen', 'mountain', 'lion', 'intens', 'kind', 'annoy', 'jason', 'priest', 'rol', 'film', 'rebel', 'shockjock', 'tv', 'movy', 'despit', 'good', 'thing', 'soundtrack', 'atrocy', 'howev', 'nic', 'see', 'tucson', 'arizon', 'promin', 'feat', 'film']


In [3]:
#Generate n-grams
from NGrams import generate_ngrams
n = 2
X_train_ngrams = generate_ngrams(X_train, n)
X_test_ngrams = generate_ngrams(X_test, n)


# Print the n-grams
# for i, ngrams in enumerate(X_train_ngrams):
#     print(f"Sentence {i+1} {n}-grams: {ngrams}")



In [7]:
import pandas as pd
import numpy as np
from termfrequency import collect_vocabulary, get_terms, vectorize, calculate_idfs, vectorize_idf

# get term frequency for the tokens

doc_terms = {}
for doc_id in range(len(X_train)):
    doc_terms[doc_id] = get_terms(X_train[doc_id]# apply to the content of the document with id doc_id
                                  )
    
# collect the vocabulary from the training set
allvocab = collect_vocabulary(X_train)
print(allvocab)

doc_vectors = vectorize(doc_terms,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
                        )
print(doc_terms[0])
print(f"{len(doc_vectors)} document vectors")
d1460_vector = doc_vectors.get(2)
print(f"{len(d1460_vector)} terms in this document") 

doc_idfs = calculate_idfs(allvocab, doc_terms# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
                        )
print(f"{len(doc_idfs)} terms with idf scores") # This should be the same number as before
print("Idf score for the word seem:")
print(doc_idfs.get("seem"))

doc_vectors = vectorize_idf(doc_terms, doc_idfs, allvocab)
print(f"{len(doc_vectors)} document vectors") # This should be the same number as before
print("Number of idf-scored words in a particular document:")
print(len(doc_vectors.get(1460)))


df_Train = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocab)

# Ensure the DataFrame is in the correct order
df_Train = df_Train.sort_index(axis=1)

df_Train


# do the same for the test set
doc_terms_test = {}
for doc_id in range(len(X_test)):
    doc_terms_test[doc_id] = get_terms(X_test[doc_id]# apply to the content of the document with id doc_id
                                  )
    
# collect the vocabulary from the training set
allvocab = collect_vocabulary(X_test)
print(allvocab)
doc_vectors = vectorize(doc_terms_test,allvocab # apply vectorize to the doc_terms and the shared vocabulary all_terms
                        )
print(doc_terms_test[0])
print(f"{len(doc_vectors)} document vectors")
d1460_vector = doc_vectors.get(2)
print(f"{len(d1460_vector)} terms in this document") 

doc_idfs = calculate_idfs(allvocab, doc_terms_test# apply calculate_idfs to the shared vocabulary all_terms and to doc_terms
                        )
print(f"{len(doc_idfs)} terms with idf scores") # This should be the same number as before
print("Idf score for the word seem:")
print(doc_idfs.get("seem"))

doc_vectors = vectorize_idf(doc_terms_test, doc_idfs, allvocab)
print(f"{len(doc_vectors)} document vectors") # This should be the same number as before
print("Number of idf-scored words in a particular document:")
print(len(doc_vectors.get(125)))


df_test = pd.DataFrame.from_dict(doc_vectors, orient='index', columns=allvocab)

# Ensure the DataFrame is in the correct order
df_test = df_test.sort_index(axis=1)

# Ensure df_test has the same columns as df_Train
# Remove columns not in df_Train
df_test = df_test.loc[:, df_test.columns.isin(df_Train.columns)]

# Add missing columns to df_test and set their values to 0
missing_cols = [col for col in df_Train.columns if col not in df_test.columns]
missing_df = pd.DataFrame(0, index=df_test.index, columns=missing_cols)
df_test = pd.concat([df_test, missing_df], axis=1)

# Reorder columns to match df_Train
df_test = df_test.reindex(columns=df_Train.columns)

df_test


# Train the MultinomialNB model
# model = MultinomialNB()
# model.fit(X_train_matrix, y_train)

# # Predict on the test set
# y_pred = model.predict(X_test_matrix)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")









# get term frequency for the tokens
# terms = get_terms(X_train)
# termFrequencies = get_term_frequency(X_train)
# df = pd.DataFrame(termFrequencies, columns=list(terms)) 
# df_normalized = df.div(np.sqrt(np.square(df).sum(axis=1)), axis=0) # L2 normalization

# # get term frequency for ngrams 
# # testngrams = X_train_ngrams[:20]
# terms2 = get_terms(X_train_ngrams)
# termFrequencies2 = get_term_frequency(X_train_ngrams)
# df2 = pd.DataFrame(termFrequencies2, columns=list(terms2))
# df_normalized2 = df2.div(np.sqrt(np.square(df2).sum(axis=1)), axis=0) # L2 normalization

# # df_normalized
# # df_normalized2

# # Combine the two arrays
# combined_df = pd.concat([df_normalized, df_normalized2], axis=1)

# # change column headings to string
# combined_df.columns = combined_df.columns.map(str)


# # Sort the columns alphabetically
# Training_combined_df = combined_df.reindex(sorted(combined_df.columns), axis=1)

# # Display the combined and sorted DataFrame
# # Training_combined_df


# terms = get_terms(X_test)
# termFrequencies = get_term_frequency(X_test)
# df = pd.DataFrame(termFrequencies, columns=list(terms)) 
# df_normalized = df.div(np.sqrt(np.square(df).sum(axis=1)), axis=0)

# terms2 = get_terms(X_test_ngrams)
# termFrequencies2 = get_term_frequency(X_test_ngrams)
# df2 = pd.DataFrame(termFrequencies2, columns=list(terms2))
# df_normalized2 = df2.div(np.sqrt(np.square(df2).sum(axis=1)), axis=0)


# # Combine the two arrays
# combined_df = pd.concat([df_normalized, df_normalized2], axis=1)

# # change column headings to string
# combined_df.columns = combined_df.columns.map(str)


# # Sort the columns alphabetically
# Testing_combined_df = combined_df.reindex(sorted(combined_df.columns), axis=1)

# # Display the combined and sorted DataFrame
# # Testing_combined_df

# Testing_combined_df = Testing_combined_df.loc[:, Testing_combined_df.columns.isin(Training_combined_df.columns)]

# # Add columns that are in Training_combined_df but not in Testing_combined_df, and set their values to 0
# for col in Training_combined_df.columns:
#     if col not in Testing_combined_df.columns:
#         Testing_combined_df[col] = 0

# # Reorder columns to match Training_combined_df
# Testing_combined_df = Testing_combined_df.reindex(columns=Training_combined_df.columns)

# Training_combined_df
# Testing_combined_df


['', '\tas', '\tfirst', '\tjason', '\tmaking', '\tthe', '0', '0001', '007', '0080', '01', '010', '0110', '03', '03oct2009', '05', '0510', '06', '08', '09', '0br', '0r', '0stars', '1', '10', '100', '1000', '10000', '100000', '1000lb', '1001', '100b', '100m', '101', '1010', '1010br', '1013', '101499', '102', '103', '1035', '104', '105', '1072000', '109', '10br', '10dirbrad', '10dirmick', '10pm', '10s', '10th', '10the', '11', '110', '1100th', '1110', '112', '112001sandler', '116', '11br', '11th', '12', '120', '1201', '121566', '123', '123000000', '125', '1282001', '12a', '12br', '12foot', '12minute', '12th', '12year', '12yearold', '13', '130', '1300', '1318', '135', '13br', '13th', '13thi', '13yearold', '14', '140', '1408br', '140am', '145150', '14a', '14goingon9', '14th', '14x', '14year', '14yearold', '14yrold', '15', '150', '150m', '1510', '1516', '153', '1547', '156', '15br', '15minute', '15th', '15to18', '15year', '16', '160', '1600', '160page', '165', '1661', '168', '16ieme', '16mm',

,,\tas,\tfirst,\tjason,\tmaking,\tthe,0,0001,007,0080,...,making,which,à,álex,álvaro,ángel,åge,écran,émigré,østbye
0,0.000000,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.000000,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.000000,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2.457351,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.000000,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396,0.000000,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
397,0.491470,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,0.000000,0,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# combining feature matrices
# from PoSTagging import combine_matrices
# df_Train = combine_matrices(df_Train, postagstrain)
# df_test = combine_matrices(df_test, postagstest)

In [8]:
#training the model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from NaiveBayes import create_classifier

#to run on my own model
# model = create_classifier(df_Train.values, y_train)
# y_pred_test = model.predict(df_test.values)
# accuracy_test = accuracy_score(y_test, y_pred_test)
# print(f"Test Accuracy: {accuracy_test}")

#to run on the sklearn model
model = MultinomialNB()
model.fit(df_Train, y_train)
y_pred_test = model.predict(df_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {accuracy_test}")


Test Accuracy: 0.815
